In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from urllib.error import HTTPError
from urllib.error import URLError

#データフレームを用意
df_result = pd.DataFrame()

#試合のIDのようなもの

#08/09日はここからスタート
#夜のうちにできれば理想的

#days = list(range(2021005421,2021005422))
#days = list(range(2021005421,2021005521))これは最初の100試合(これは完了)
#days = list(range(2021005521,2021005621))#これは次の100試合(これは完了)
#days = list(range(2021005621,2021005821))#これは次の200試合(これは完了)
#days = list(range(2021005821,2021005921))#これは次の100試合(これは完了)
#days = list(range(2021005921,2021006021))#これは次の100試合(完了)
#days = list(range(2021006021,2021006121))#100試合分(完了)
#days = list(range(2021006121,2021006221))#100試合分(完了)
#days = list(range(2021006221,2021006279))#58試合分(完了)
days = list(range(2021011400,2021011465))#16試合分(未完了)

#days = list(range(2021006221,2021006279))
#2021006278が9/28までの最新試合
#2021005421が2022年の開幕戦(3/25(金)ドラゴンズ対ジャイアンツ)
#2021005325が2022年の初めの試合のID(2/26(土)ソフトバンク対オリックス)
#2021006003が2022年7月24日時点で最新の試合(西武対楽天)
#678試合分のデータがある
#days = ['2021005991']
for d in days:
    url = 'https://baseball.yahoo.co.jp/npb/game/' + str(d) + '/score?index=0110100'
    
    for i in range(1000):
        #1秒待機
        time.sleep(1)
        
        #read_htmlでデータを取得
        try:
            df = pd.read_html(url, encoding='utf-8')
        except:
            break
        
        #BeautifulSoupでデータを取得
        try:
            r = requests.get(url)
            soup = BeautifulSoup(r.content, 'html.parser')
        except:
            break
        
        #「次へ」ボタンからURLを取得
        try:
            next = soup.find('a', id= 'btn_next').get('href')
        except:
            break
            
        
        if len(df) == 19: 
            #投球数、球種、球速、結果、投手、打者、スコアを取得
            result = df[14].drop('投球数.1', axis=1)
            result['投手'] = df[13].loc[0, '投手']
            result['打者'] = df[13].loc[0, '打者']
            result['スコア'] = '-'.join(map(str, df[1].iloc[:, 1].tolist()))
            
            #アウトカウントをスクレイピングする
            sbo_class = soup.find("div", class_= "sbo")
            out_count = sbo_class.find("p", class_="o")
            out_count = out_count.find('b')
            if out_count is not None:
                result['アウトカウント'] = out_count.text
            else:
                result['アウトカウント'] = ""
            #print(out_count)確認のためのコード
            #print(result['アウトカウント'])確認のためのコード
            
            #投球コースを取得
            #Course = soup.find('div', class_= 'bb-allocationChart').find_all('span', class_='bb-icon__ballCircle')
            #course = [c['style'] for c in Course]
            #result['投球コース'] = course

            #データフレームを結合
            df_result = pd.concat([df_result, result])

        #次のページのURLに更新
        url = 'https://baseball.yahoo.co.jp' + next

In [6]:
df_result

,投球数,球種,球速,結果,投手,打者,スコア,アウトカウント
0,1,ストレート,141km/h,見逃し,石田 健大,岡林 勇希,0-0,
1,2,スライダー,129km/h,ボール,石田 健大,岡林 勇希,0-0,
2,3,ストレート,143km/h,ボール,石田 健大,岡林 勇希,0-0,
3,4,スライダー,127km/h,ボール,石田 健大,岡林 勇希,0-0,
4,5,スライダー,124km/h,遊失 [送球できず],石田 健大,岡林 勇希,0-0,
...,...,...,...,...,...,...,...,...
3,4,ストレート,150km/h,ファウル,R.マルティネス,西川 龍馬,3-0,●●●
4,5,ストレート,152km/h,ボール,R.マルティネス,西川 龍馬,3-0,●●●
5,6,ストレート,-,ファウル,R.マルティネス,西川 龍馬,3-0,●●●
6,7,ストレート,-,ファウル,R.マルティネス,西川 龍馬,3-0,●●●


In [7]:
df_result.to_csv('data/8_game_results_859_874.csv',index=False)

In [ ]:
"""
try:
    html = urlopen("http://www.sakutyuu.com1/")
except HTTPError as e:
    #エラーの場合の処理
    print(e)
except URLError as e:
    #エラーの場合の処理
    print("URLに接続するが処理できない")
else:
"""